# Modelo de Clasificación Local de Extremo a Extremo

Este notebook implementa un pipeline completo de Machine Learning de forma local:

1.  **Carga de Datos**: Usa un dataset de texto real (`20 Newsgroups`) de forma local.
2.  **Análisis y Tokenización**: Analiza la longitud de los textos usando un tokenizador de BERT.
3.  **Generación de Embeddings**: Convierte el texto en vectores numéricos (embeddings) usando un modelo BERT pre-entrenado.
4.  **Entrenamiento Multi-Modelo**: Entrena y optimiza tres modelos (XGBoost, MLP con PyTorch, Regresión Logística) usando **Optuna**.
5.  **Creación de Ensemble**: Combina los tres modelos en un **ensemble ponderado** para mejorar la precisión.
6.  **Evaluación**: Evalúa el rendimiento del modelo ensemble final.

Todo el proceso se ejecuta localmente sin dependencias de la nube, aprovechando la GPU si está disponible.

## 1. Instalación y Configuración

In [ ]:
#!pip install transformers torch datasets scikit-learn xgboost pandas seaborn matplotlib tqdm optuna

In [18]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [ ]:
# --- CONFIGURACIÓN GENERAL ---
import pandas as pd
import numpy as np
import os
import pickle
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import time
import torch
import xgboost as xgb
from tqdm.auto import tqdm

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, log_loss, f1_score

# --- Parámetros de Configuración ---
BERT_MODEL_NAME = 'bert-base-uncased'
MAX_SAMPLES = 2500 # Limitar el número de muestras para que la ejecución sea más rápida. Poner a None para usar el dataset completo.
MAX_TOKEN_LENGTH = 128 # Max longitud para truncar/rellenar tokens.

# --- Configuración de Dispositivo (GPU o CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# --- Definición de Rutas Locales ---
job_id = f"local-ensemble-job-{int(time.time())}"
BASE_DIR = "datos_locales"
INPUT_DIR = os.path.join(BASE_DIR, "input")
PROCESSED_DIR = os.path.join(BASE_DIR, "processed", job_id)
MODEL_OUTPUT_DIR = os.path.join(BASE_DIR, "model_output", job_id)

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

INPUT_EMBEDDINGS_FILENAME = "text_embeddings.csv"
LOCAL_EMBEDDINGS_PATH = os.path.join(INPUT_DIR, INPUT_EMBEDDINGS_FILENAME)

print(f"\nID de trabajo para esta ejecución: {job_id}")
print(f"Ruta para embeddings generados: {LOCAL_EMBEDDINGS_PATH}")

## 2. Carga, Análisis y Tokenización de Datos

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from tqdm.auto import tqdm
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print("Cargando dataset '20 Newsgroups' desde Scikit-learn (100% offline)...")
# NOTA: La primera vez que se ejecute, scikit-learn puede descargar y guardar los datos en caché.
# Después de eso, siempre se cargará localmente.

# Cargamos tanto el conjunto de entrenamiento como el de prueba para tener más datos
try:
    # El parámetro 'remove' limpia los metadatos para que el modelo se centre en el contenido del texto.
    newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
    newsgroups_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
except Exception as e:
    print(f"\nERROR: No se pudo cargar el dataset '20 Newsgroups'. Causa: {e}")
    print("Si es un error de red, ejecuta este notebook una vez en una máquina con internet para que se descargue y guarde en caché.")
    raise

# Combinamos los datos en un solo DataFrame de pandas
all_text = newsgroups_train.data + newsgroups_test.data
all_targets = list(newsgroups_train.target) + list(newsgroups_test.target)
label_names = newsgroups_train.target_names
all_label_names = [label_names[i] for i in all_targets]

df = pd.DataFrame({
    'text': all_text,
    'label_name': all_label_names
})

# Tomar una muestra si se especificó para acelerar el proceso
if 'MAX_SAMPLES' in locals() and MAX_SAMPLES is not None:
    print(f"Tomando una muestra aleatoria de {MAX_SAMPLES} registros.")
    df = df.sample(n=MAX_SAMPLES, random_state=42).reset_index(drop=True)

print(f"Dataset cargado con {len(df)} filas.")
print("\nPrimeras 5 filas:")
print(df.head())

print("\nDistribución de clases (en la muestra):")
print(df['label_name'].value_counts())

# Cargar tokenizador de BERT (esto todavía necesita internet la primera vez que se ejecuta)
print(f"\nCargando tokenizador: {BERT_MODEL_NAME}")
try:
    tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
except (ConnectionError, OSError) as e:
    print(f"\nERROR: No se pudo descargar el tokenizador. Causa: {e}")
    print("Si el problema es de red, descarga la carpeta del modelo 'bert-base-uncased' manualmente desde el Hub y cárgalo desde la ruta local.")
    raise

# Medir longitud de tokens
print("Analizando longitud de los textos en tokens...")
# Usamos tqdm para ver una barra de progreso, ya que puede tardar un poco
df['token_length'] = [len(tokenizer.encode(text, max_length=512, truncation=True)) for text in tqdm(df['text'])]

# Visualizar la distribución de la longitud de tokens
plt.figure(figsize=(10, 6))
sns.histplot(df['token_length'], bins=50, kde=True)
plt.title('Distribución de la Longitud de Tokens por Texto')
plt.xlabel('Longitud de Tokens')
plt.ylabel('Frecuencia')
# Usamos MAX_TOKEN_LENGTH definido en la primera celda
plt.axvline(x=MAX_TOKEN_LENGTH, color='r', linestyle='--', label=f'Max Length = {MAX_TOKEN_LENGTH}')
plt.legend()
plt.show()

print(f"Longitud promedio de tokens: {df['token_length'].mean():.2f}")

## 3. Generación de Embeddings con BERT
Este es el paso más intensivo computacionalmente. Se recomienda usar una GPU.

In [ ]:
print(f"Cargando modelo pre-entrenado: {BERT_MODEL_NAME}")
model = AutoModel.from_pretrained(BERT_MODEL_NAME).to(device)
model.eval() # Poner el modelo en modo de evaluación

def get_bert_embeddings(batch_text):
    """Tokeniza un lote de texto y obtiene el embedding [CLS] de BERT."""
    inputs = tokenizer(batch_text, padding=True, truncation=True, 
                       max_length=MAX_TOKEN_LENGTH, return_tensors='pt')
    
    # Mover tensores al dispositivo (GPU/CPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Usamos el embedding del token [CLS] (índice 0)
    cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embeddings

print("\nGenerando embeddings... Esto puede tardar varios minutos.")
batch_size = 32
all_embeddings = []

# tqdm ofrece una barra de progreso
for i in tqdm(range(0, len(df), batch_size)):
    batch_df = df.iloc[i:i+batch_size]
    batch_text = batch_df['text'].tolist()
    embeddings = get_bert_embeddings(batch_text)
    all_embeddings.append(embeddings)

# Combinar todos los embeddings de los lotes
final_embeddings = np.vstack(all_embeddings)

# Crear un DataFrame con los embeddings
embedding_cols = [f'dim_{i}' for i in range(final_embeddings.shape[1])]
df_embeddings = pd.DataFrame(final_embeddings, columns=embedding_cols)

# Unir los embeddings con las etiquetas originales
# Usamos 'label_name' como la etiqueta de texto que queremos predecir
df_final = pd.concat([df[['label_name']].rename(columns={'label_name': 'label'}), df_embeddings], axis=1)

# Guardar el resultado para los siguientes pasos
df_final.to_csv(LOCAL_EMBEDDINGS_PATH, index=False)

print(f"\n✓ Embeddings generados y guardados en: {LOCAL_EMBEDDINGS_PATH}")
print("Dimensiones del DataFrame final:", df_final.shape)
print(df_final.head())

## 4. División y Codificación de Datos

In [ ]:
print("\n--- Ejecutando Lógica de División y Codificación ---")

# 1. Cargar los datos de entrada (embeddings generados)
print(f"Cargando datos desde {LOCAL_EMBEDDINGS_PATH}")
df = pd.read_csv(LOCAL_EMBEDDINGS_PATH)
print(f"Datos cargados. {len(df)} filas.")

# 2. Codificar las etiquetas
label_col_name = 'label'
encoded_label_col = f"{label_col_name}_encoded"
embedding_cols = [col for col in df.columns if col.startswith('dim_')]

print(f"Columnas de embedding detectadas: {len(embedding_cols)}")
print(f"Columna de etiquetas: {label_col_name}")

# La lógica de filtrar clases con <10 instancias se mantiene por robustez
label_counts = df[label_col_name].value_counts()
valid_labels = label_counts[label_counts > 9].index
df = df[df[label_col_name].isin(valid_labels)].reset_index(drop=True)

print(f"Filtrado completado. Se conservaron {len(valid_labels)} clases con más de 9 muestras.")

print("Codificando etiquetas...")
label_encoder = LabelEncoder()
df[encoded_label_col] = label_encoder.fit_transform(df[label_col_name])
num_classes = len(label_encoder.classes_)
print(f"Se detectaron y codificaron {num_classes} clases.")
print("Mapeo de etiquetas:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

X = df[embedding_cols].values
y = df[encoded_label_col].values

# 3. Dividir los datos
print("Dividiendo los datos en conjuntos de entrenamiento, validación y prueba...")
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval)
print(f"Tamaño Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")

# Guardar el codificador y los datos de prueba para la evaluación final
label_encoder_path = os.path.join(MODEL_OUTPUT_DIR, "label_encoder.pkl")
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)

test_features_path = os.path.join(PROCESSED_DIR, "test_features.pkl")
test_labels_path = os.path.join(PROCESSED_DIR, "test_labels.pkl")
with open(test_features_path, 'wb') as f: pickle.dump(X_test, f)
with open(test_labels_path, 'wb') as f: pickle.dump(y_test, f)

print(f"\n✓ Procesamiento completado exitosamente!")

## 5. Entrenamiento Multi-Modelo con Optuna

En esta sección, optimizaremos tres modelos diferentes usando Optuna para encontrar los mejores hiperparámetros. Luego, los combinaremos en un ensemble ponderado para maximizar el rendimiento.

In [ ]:
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

print("--- Preparando datos para el entrenamiento ---")
# Algunos modelos (MLP, Regresión Logística) se benefician del escalado de características.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convertimos los datos de validación a tensores para PyTorch una sola vez
X_val_torch = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
y_val_torch = torch.tensor(y_val, dtype=torch.long).to(device)

print("Datos escalados y tensores de PyTorch listos.")

### 5.1 Definición de las Funciones Objetivo para Optuna

Cada función `objective` define cómo Optuna debe entrenar y evaluar un modelo para un conjunto de hiperparámetros (`trial`). El objetivo es minimizar la métrica `log_loss` en el conjunto de validación.

In [ ]:
# --- 1. Objetivo para MLP con PyTorch ---
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, activation_fn, dropout_rate):
        super(MLP, self).__init__()
        layers = []
        current_size = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(activation_fn())
            layers.append(nn.Dropout(dropout_rate))
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def objective_mlp(trial):
    # Hiperparámetros a optimizar
    n_layers = trial.suggest_int('n_layers', 1, 3)
    hidden_layers = [trial.suggest_int(f'n_units_l{i}', 64, 512) for i in range(n_layers)]
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    activation_name = trial.suggest_categorical('activation', ['ReLU', 'Tanh'])
    activation_fn = getattr(nn, activation_name)
    
    # Modelo, datos y lógica se mueven al dispositivo (GPU/CPU)
    model = MLP(X_train_scaled.shape[1], hidden_layers, num_classes, activation_fn, dropout_rate).to(device)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    train_dataset = TensorDataset(torch.tensor(X_train_scaled, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    best_val_loss = float('inf')
    patience, patience_counter = 5, 0

    for epoch in range(50):
        model.train()
        for data, target in train_loader:
            # Mover cada lote al dispositivo
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            # El tensor de validación ya está en el dispositivo
            val_outputs = model(X_val_torch)
            val_loss = criterion(val_outputs, y_val_torch).item()
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience: break
        
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss

# --- 2. Objetivo para XGBoost (GPU Habilitado) ---
def objective_xgboost(trial):
    params = {
        'objective': 'multi:softprob',
        'num_class': num_classes,
        'eval_metric': 'mlogloss',
        'verbosity': 0,
        'seed': 42,
        # Configuración explícita para usar GPU si está disponible
        'tree_method': 'hist', 
        'device': 'cuda' if device.type == 'cuda' else 'cpu',
        # Hiperparámetros
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10, step=1),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }
    
    model = xgb.XGBClassifier(**params, early_stopping_rounds=15, use_label_encoder=False)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    y_pred_proba = model.predict_proba(X_val)
    return log_loss(y_val, y_pred_proba)

# --- 3. Objetivo para Regresión Logística (CPU Paralelizado) ---
def objective_logistic(trial):
    # Nota: Este modelo se ejecuta en CPU. 'n_jobs=-1' usa todos los núcleos.
    params = {
        'C': trial.suggest_float('C', 1e-4, 1e2, log=True),
        'solver': 'saga',
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet']),
        'max_iter': 1000,
        'random_state': 42,
        'multi_class': 'multinomial',
        'n_jobs': -1 # Usar todos los núcleos de CPU disponibles
    }
    if params['penalty'] == 'elasticnet':
        params['l1_ratio'] = trial.suggest_float('l1_ratio', 0.0, 1.0)
    
    model = LogisticRegression(**params)
    model.fit(X_train_scaled, y_train)
    y_pred_proba = model.predict_proba(X_val_scaled)
    return log_loss(y_val, y_pred_proba)

print(f"Funciones objetivo de Optuna definidas. Dispositivo de cómputo principal: {device}")

### 5.2 Ejecución de la Búsqueda de Hiperparámetros
Lanzamos los estudios de Optuna para cada modelo. Esto puede tardar un tiempo.

In [ ]:
models_config = {
    'XGBoost': {'objective_func': objective_xgboost, 'n_trials': 40},
    'MLP_PyTorch': {'objective_func': objective_mlp, 'n_trials': 50},
    'LogisticRegression': {'objective_func': objective_logistic, 'n_trials': 30}
}

model_results = {}

for model_name, config in models_config.items():
    print(f"\n--- Optimizando {model_name} ---")
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    )
    study.optimize(config['objective_func'], n_trials=config['n_trials'], show_progress_bar=True)
    
    model_results[model_name] = {
        'best_params': study.best_params,
        'best_score': study.best_value
    }
    print(f"✓ {model_name} completado. Mejor LogLoss: {study.best_value:.4f}")
    print(f"  Mejores parámetros: {study.best_params}")

### 5.3 Entrenamiento de los Modelos Finales

Ahora entrenamos cada modelo una última vez usando el conjunto completo de entrenamiento (`X_train`, `y_train`) y los mejores hiperparámetros encontrados.

In [ ]:
trained_models = {}

print(f"--- Entrenando modelos finales con los mejores hiperparámetros en el dispositivo: {device} ---")

# 1. XGBoost (Entrenamiento final en GPU)
print("Entrenando XGBoost final en GPU...")
xgb_params = model_results['XGBoost']['best_params']
final_xgb = xgb.XGBClassifier(
    objective='multi:softprob', num_class=num_classes, eval_metric='mlogloss',
    seed=42, use_label_encoder=False,
    # Configuración explícita para GPU
    tree_method='hist',
    device='cuda' if device.type == 'cuda' else 'cpu',
    early_stopping_rounds=15,
    **xgb_params
)
final_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
trained_models['XGBoost'] = final_xgb
print("✓ Modelo XGBoost final entrenado.")

# 2. Regresión Logística (Entrenamiento final en CPU)
print("\nEntrenando Regresión Logística final en CPU (paralelizado)...")
log_params = model_results['LogisticRegression']['best_params']
if 'l1_ratio' not in log_params and log_params.get('penalty') == 'elasticnet':
    log_params['l1_ratio'] = 0.5
final_log = LogisticRegression(
    solver='saga', max_iter=2000, random_state=42, multi_class='multinomial', 
    n_jobs=-1, # Usar todos los núcleos de CPU
    **log_params
)
final_log.fit(X_train_scaled, y_train)
trained_models['LogisticRegression'] = final_log
print("✓ Modelo de Regresión Logística final entrenado.")


# 3. MLP con PyTorch (Entrenamiento final en GPU)
print("\nEntrenando MLP (PyTorch) final en GPU...")
mlp_params = model_results['MLP_PyTorch']['best_params']
hidden_layers = [mlp_params[f'n_units_l{i}'] for i in range(mlp_params['n_layers'])]
# Mover el modelo al dispositivo (GPU/CPU)
final_mlp = MLP(
    input_size=X_train_scaled.shape[1],
    hidden_layers=hidden_layers,
    output_size=num_classes,
    activation_fn=getattr(nn, mlp_params['activation']),
    dropout_rate=mlp_params['dropout_rate']
).to(device)

optimizer = getattr(optim, mlp_params['optimizer'])(final_mlp.parameters(), lr=mlp_params['lr'])
criterion = nn.CrossEntropyLoss()
train_dataset = TensorDataset(torch.tensor(X_train_scaled, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

for epoch in tqdm(range(60), desc="Epochs MLP final"):
    final_mlp.train()
    for data, target in train_loader:
        # Mover cada lote a la GPU
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = final_mlp(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
trained_models['MLP_PyTorch'] = final_mlp
print("✓ Modelo MLP (PyTorch) final entrenado.")

print("\n✓ Todos los modelos finales han sido entrenados.")

### 5.4 Creación y Evaluación del Ensemble Ponderado

Definimos un clasificador `Ensemble` que combina las predicciones de los modelos. Los pesos se calculan en base al rendimiento de cada modelo en el conjunto de validación: un menor `log_loss` resulta en un mayor peso.

In [ ]:
class WeightedEnsembleClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, models, weights, scaler=None, device_str='cpu'):
        self.models = models
        self.weights = weights
        self.scaler = scaler
        self.device = torch.device(device_str)
        self.classes_ = None
        
        # Verificar si CuPy está disponible para el uso de la GPU en XGBoost
        self.gpu_available = False
        if self.device.type == 'cuda':
            try:
                import cupy as cp
                self.gpu_available = True
            except ImportError:
                self.gpu_available = False
    
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        return self

    def predict_proba(self, X):
        ensemble_proba = np.zeros((X.shape[0], len(self.classes_)))
        X_scaled = self.scaler.transform(X) if self.scaler else X

        for name, model in self.models.items():
            if name == 'XGBoost':
                if self.gpu_available:
                    import cupy as cp
                    X_gpu = cp.asarray(X)
                    proba = cp.asnumpy(model.predict_proba(X_gpu))
                else:
                    proba = model.predict_proba(X)
            elif name in ['MLP_Scikit', 'LogisticRegression']:
                proba = model.predict_proba(X_scaled)
            
            ensemble_proba += proba * self.weights[name]
        return ensemble_proba

    def predict(self, X):
        probas = self.predict_proba(X)
        return self.classes_[np.argmax(probas, axis=1)]

# --- CÓDIGO CORREGIDO ---

# CORRECCIÓN: Volver a verificar la disponibilidad de GPU al inicio de la celda
gpu_available = False
if device.type == 'cuda':
    try:
        import cupy as cp
        gpu_available = True
    except ImportError:
        gpu_available = False


print("--- Evaluando modelos individuales en el conjunto de validación ---")
individual_metrics = {}
X_val_scaled = scaler.transform(X_val)

for name, model in trained_models.items():
    if name == 'XGBoost':
        if gpu_available:
            import cupy as cp
            X_val_gpu = cp.asarray(X_val)
            proba = cp.asnumpy(model.predict_proba(X_val_gpu))
            pred = cp.asnumpy(model.predict(X_val_gpu))
        else:
            proba = model.predict_proba(X_val)
            pred = model.predict(X_val)
    elif name in ['MLP_Scikit', 'LogisticRegression']:
        proba = model.predict_proba(X_val_scaled)
        pred = model.predict(X_val_scaled)
    
    loss = log_loss(y_val, proba)
    acc = accuracy_score(y_val, pred)
    f1 = f1_score(y_val, pred, average='weighted')
    
    individual_metrics[name] = {'log_loss': loss, 'accuracy': acc, 'f1_score': f1}
    print(f"Modelo: {name:<20} | LogLoss: {loss:.4f} | Accuracy: {acc:.4f}")

# Calcular pesos
losses = {name: metrics['log_loss'] for name, metrics in individual_metrics.items()}
scores = {name: 1.0 / (loss + 1e-9) for name, loss in losses.items()}
total_score = sum(scores.values())
weights = {name: score / total_score for name, score in scores.items()}

print("\n--- Pesos del Ensemble Calculados ---")
for name, w in weights.items(): print(f"{name:<20} | Peso: {w:.3f} | LogLoss (Val): {losses[name]:.4f}")

# Crear el ensemble final
ensemble_model = WeightedEnsembleClassifier(trained_models, weights, scaler, device.type)
ensemble_model.fit(X_train, y_train)

# Evaluar el ensemble
ensemble_proba = ensemble_model.predict_proba(X_val)
ensemble_pred = ensemble_model.predict(X_val)
ensemble_log_loss = log_loss(y_val, ensemble_proba)
ensemble_accuracy = accuracy_score(y_val, ensemble_pred)
ensemble_f1 = f1_score(y_val, ensemble_pred, average='weighted')

print("\n--- Comparación de Rendimiento (Validación) ---")
print(f"{'Modelo':<20} | {'Log Loss':<10} | {'Accuracy':<10} | {'F1-Score':<10}")
print("="*60)
for name, metrics in individual_metrics.items():
    print(f"{name:<20} | {metrics['log_loss']:<10.4f} | {metrics['accuracy']:<10.4f} | {metrics['f1_score']:<10.4f}")
print("—"*60)
print(f"{'🏆 ENSEMBLE':<20} | {ensemble_log_loss:<10.4f} | {ensemble_accuracy:<10.4f} | {ensemble_f1:<10.4f}")

## 6. Evaluación Final del Modelo Ensemble (en Conjunto de Prueba)

In [ ]:
print("\n--- Ejecutando Evaluación Final del Ensemble en el Conjunto de Prueba ---")

# 1. Cargar los datos de prueba y el codificador de etiquetas
with open(test_features_path, 'rb') as f: X_test_eval = pickle.load(f)
with open(test_labels_path, 'rb') as f: y_test_eval = pickle.load(f)
with open(label_encoder_path, 'rb') as f: label_encoder_eval = pickle.load(f)
print(f"Datos de prueba cargados: {X_test_eval.shape[0]} muestras.")

# 2. Obtener predicciones del modelo ensemble
y_pred_eval = ensemble_model.predict(X_test_eval)

# 3. Calcular y mostrar métricas
print("\n--- Resultados de la Evaluación Final ---")
acc = accuracy_score(y_test_eval, y_pred_eval)
report = classification_report(y_test_eval, y_pred_eval, target_names=label_encoder_eval.classes_)
cm = confusion_matrix(y_test_eval, y_pred_eval)

print(f"Accuracy en el conjunto de prueba: {acc:.4f}\n")
print("Reporte de Clasificación:")
print(report)

print("\nMatriz de Confusión:")
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder_eval.classes_, yticklabels=label_encoder_eval.classes_)
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.title('Matriz de Confusión del Modelo Ensemble (Prueba)')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n--- Evaluación Completada ---")

## 7. Guardado de Artefactos del Modelo

Guardamos el modelo ensemble final, los modelos individuales, el escalador y los resultados de la optimización para su uso futuro.

In [ ]:
print(f"--- Guardando artefactos en {MODEL_OUTPUT_DIR} ---")

# 1. Guardar el modelo ensemble completo
ensemble_path = os.path.join(MODEL_OUTPUT_DIR, "ensemble_model.pkl")
with open(ensemble_path, 'wb') as f:
    pickle.dump(ensemble_model, f)
print(f"✓ Modelo ensemble guardado en: {ensemble_path}")

# 2. Guardar los modelos individuales
# Para PyTorch, guardamos el state_dict, que es más robusto
mlp_path = os.path.join(MODEL_OUTPUT_DIR, "mlp_pytorch.pth")
torch.save(trained_models['MLP_PyTorch'].state_dict(), mlp_path)
print(f"✓ Modelo MLP (PyTorch) guardado en: {mlp_path}")

for name, model in trained_models.items():
    if name != 'MLP_PyTorch':
        model_path = os.path.join(MODEL_OUTPUT_DIR, f"{name.lower()}_model.pkl")
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        print(f"✓ Modelo {name} guardado en: {model_path}")

# 3. Guardar el escalador
scaler_path = os.path.join(MODEL_OUTPUT_DIR, "scaler.pkl")
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ Scaler guardado en: {scaler_path}")

# 4. Guardar los resultados de Optuna
results_path = os.path.join(MODEL_OUTPUT_DIR, "optuna_results.pkl")
with open(results_path, 'wb') as f:
    pickle.dump(model_results, f)
print(f"✓ Resultados de Optuna guardados en: {results_path}")

print("\n🎉 Pipeline completado y todos los artefactos han sido guardados.")